# Import packages and cleaned dataset

In [1]:
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin

In [2]:
supplyChain = pd.read_csv("../../data/raw/Q1_2015.csv", encoding='ISO-8859-1')
future_data = pd.read_csv("../../data/raw/future_data.csv", encoding='ISO-8859-1')

# Data Cleaning and Preprocessing

In [3]:
supplyChain_clean = supplyChain.drop(columns=['Days for shipping (real)', 'Delivery Status', 'Late_delivery_risk', 
                                        'shipping date (DateOrders)', 'Benefit per order', 'Sales per customer', 'Category Id',
                                        'Order Profit Per Order', 'Order Item Discount', 'Order Item Total', 'Order Status', 
                                        'Customer Email', 'Customer Password', 'Latitude', 'Longitude', 'Product Description', 'Product Image',
                                        'Customer Fname', 'Customer Id', 'Customer Lname', 'Department Id',
                                             'Order Customer Id', 'Order Item Cardprod Id', 'Order Item Id',
                                             'Product Card Id', 'Product Category Id', 'Order Id', 'Customer Street',
                                             'Customer Zipcode', 'Order Zipcode', 'Order Item Product Price',
                                             'Product Price', 'Order Item Profit Ratio', 'Product Status'])
future_data = future_data.drop(columns=['Days for shipping (real)', 'Delivery Status', 'Late_delivery_risk', 
                                        'shipping date (DateOrders)', 'Benefit per order', 'Sales per customer', 'Category Id',
                                        'Order Profit Per Order', 'Order Item Discount', 'Order Item Total', 'Order Status', 
                                        'Customer Email', 'Customer Password', 'Latitude', 'Longitude', 'Product Description', 'Product Image',
                                        'Customer Fname', 'Customer Id', 'Customer Lname', 'Department Id',
                                             'Order Customer Id', 'Order Item Cardprod Id', 'Order Item Id',
                                             'Product Card Id', 'Product Category Id', 'Order Id', 'Customer Street',
                                             'Customer Zipcode', 'Order Zipcode', 'Order Item Product Price',
                                             'Product Price', 'Order Item Profit Ratio', 'Product Status'])

In [4]:
supplyChain_clean.columns

Index(['Type', 'Days for shipment (scheduled)', 'Category Name',
       'Customer City', 'Customer Country', 'Customer Segment',
       'Customer State', 'Department Name', 'Market', 'Order City',
       'Order Country', 'order date (DateOrders)', 'Order Item Discount Rate',
       'Order Item Quantity', 'Sales', 'Order Region', 'Order State',
       'Product Name', 'Shipping Mode'],
      dtype='object')

In [5]:
categorical_cols = ['Type', 'Category Name', 'Customer City', 'Customer Country',
       'Customer Segment', 'Customer State', 'Department Name', 'Market',
       'Order City', 'Order Country',
       'Order Region', 'Order State', 'Product Name', 'Shipping Mode', "Year", "Month"]

In [6]:
# Date and Time Features 
supplyChain_clean['order date (DateOrders)'] = pd.to_datetime(supplyChain_clean['order date (DateOrders)']) 
#supplyChain_clean['Day of Week'] = supplyChain_clean['order date (DateOrders)'].dt.dayofweek 
supplyChain_clean['Month'] = supplyChain_clean['order date (DateOrders)'].dt.month 
supplyChain_clean['Year'] = supplyChain_clean['order date (DateOrders)'].dt.year 
#supplyChain_clean['Week of Year'] = supplyChain_clean['order date (DateOrders)'].dt.isocalendar().week 

supplyChain_clean['order date (DateOrders)'] = pd.to_datetime(supplyChain_clean['order date (DateOrders)'])

# Sorting the DataFrame by the 'order date (DateOrders)' column
supplyChain_clean.sort_values(by='order date (DateOrders)', inplace=True)

# Dropping the 'order date (DateOrders)' column
supplyChain_clean.drop(columns=['order date (DateOrders)'], inplace=True)

In [7]:
# Date and Time Features 
future_data['order date (DateOrders)'] = pd.to_datetime(future_data['order date (DateOrders)']) 
#supplyChain_clean['Day of Week'] = supplyChain_clean['order date (DateOrders)'].dt.dayofweek 
future_data['Month'] = future_data['order date (DateOrders)'].dt.month 
future_data['Year'] = future_data['order date (DateOrders)'].dt.year 
#supplyChain_clean['Week of Year'] = supplyChain_clean['order date (DateOrders)'].dt.isocalendar().week 

future_data['order date (DateOrders)'] = pd.to_datetime(future_data['order date (DateOrders)'])

# Sorting the DataFrame by the 'order date (DateOrders)' column
future_data.sort_values(by='order date (DateOrders)', inplace=True)

# Dropping the 'order date (DateOrders)' column
future_data.drop(columns=['order date (DateOrders)'], inplace=True)

In [8]:
# import pandas as pd

# Grouping the data by the specified variables
grouped_data = supplyChain_clean.groupby(['Type', 'Category Name', 'Customer City', 'Customer Country',
                                            'Customer Segment', 'Customer State', 'Department Name', 'Market',
                                            'Order City', 'Order Country', 'Order Region', 'Order State',
                                            'Product Name', 'Shipping Mode', 'Year', 'Month'])

# Applying aggregation functions to the grouped data
aggregated_data = grouped_data.agg({
    # Aggregation functions for each column
    'Sales': 'sum',
    'Days for shipment (scheduled)': 'mean', 
    'Order Item Discount Rate': 'mean',
    'Order Item Quantity': "sum"
})

# Resetting index to make the DataFrame flat
aggregated_data.reset_index(inplace=True)

In [9]:
aggregated_data

,Type,Category Name,Customer City,Customer Country,Customer Segment,Customer State,Department Name,Market,Order City,Order Country,Order Region,Order State,Product Name,Shipping Mode,Year,Month,Sales,Days for shipment (scheduled),Order Item Discount Rate,Order Item Quantity
0,CASH,Accessories,Bronx,EE. UU.,Consumer,NY,Outdoors,LATAM,Mejicanos,El Salvador,Central America,San Salvador,Team Golf Pittsburgh Steelers Putter Grip,First Class,2015,2,124.949997,1.0,0.000,5
1,CASH,Accessories,Brooklyn,EE. UU.,Home Office,NY,Outdoors,LATAM,La Romana,República Dominicana,Caribbean,La Romana,Team Golf Tennessee Volunteers Putter Grip,First Class,2015,2,74.970001,1.0,0.200,3
2,CASH,Accessories,Brownsville,EE. UU.,Corporate,TX,Outdoors,LATAM,San Salvador,El Salvador,Central America,San Salvador,Team Golf New England Patriots Putter Grip,Standard Class,2015,1,49.980000,4.0,0.250,2
3,CASH,Accessories,Buffalo,EE. UU.,Corporate,NY,Outdoors,LATAM,Santiago de los Caballeros,República Dominicana,Caribbean,Santiago de Chile,Team Golf St. Louis Cardinals Putter Grip,Standard Class,2015,1,124.949997,4.0,0.120,5
4,CASH,Accessories,Caguas,Puerto Rico,Consumer,PR,Outdoors,LATAM,Buenos Aires,Argentina,South America,Buenos Aires,Team Golf New England Patriots Putter Grip,Second Class,2015,3,74.970001,2.0,0.090,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13174,TRANSFER,Women's Apparel,West Covina,EE. UU.,Corporate,CA,Golf,LATAM,Santo Domingo,República Dominicana,Caribbean,Santo Domingo,Nike Men's Dri-FIT Victory Golf Polo,Standard Class,2015,2,200.000000,4.0,0.090,4
13175,TRANSFER,Women's Apparel,West Jordan,EE. UU.,Corporate,UT,Golf,LATAM,Lagos de Moreno,México,Central America,Jalisco,Nike Men's Dri-FIT Victory Golf Polo,Standard Class,2015,3,300.000000,4.0,0.155,6
13176,TRANSFER,Women's Apparel,West Jordan,EE. UU.,Corporate,UT,Golf,LATAM,Mejicanos,El Salvador,Central America,San Salvador,Nike Men's Dri-FIT Victory Golf Polo,Second Class,2015,2,150.000000,2.0,0.090,3
13177,TRANSFER,Women's Apparel,West Lafayette,EE. UU.,Consumer,IN,Golf,LATAM,Lima,Perú,South America,Lima (ciudad),Nike Men's Dri-FIT Victory Golf Polo,First Class,2015,1,200.000000,1.0,0.060,4


In [10]:
supplyChain_clean = aggregated_data

In [11]:
supplyChain_clean.to_csv("../../data/processed/Q1_2015_cleaned.csv", index = False)

In [12]:
# Grouping the data by the specified variables
grouped_data2 = future_data.groupby(['Type', 'Category Name', 'Customer City', 'Customer Country',
                                            'Customer Segment', 'Customer State', 'Department Name', 'Market',
                                            'Order City', 'Order Country', 'Order Region', 'Order State',
                                            'Product Name', 'Shipping Mode', 'Year', 'Month'])

# Applying aggregation functions to the grouped data
aggregated_data2 = grouped_data2.agg({
    # Aggregation functions for each column
    'Sales': 'sum',
    'Days for shipment (scheduled)': 'mean', 
    'Order Item Discount Rate': 'mean',
    'Order Item Quantity': "sum"
})

# Resetting index to make the DataFrame flat
aggregated_data2.reset_index(inplace=True)

In [13]:
future_data_cleaned = aggregated_data2

In [14]:
future_data_cleaned.to_csv("../../data/processed/future_data_cleaned.csv", index = False)

In [15]:
# supplyChain_clean_2000 = supplyChain_clean.iloc[:2000]

In [16]:
# supplyChain_clean_2000.to_csv("DataCo_cleaned.csv", index = False)

In [17]:
# supplyChain_clean_2000

# Write to Pipeline

In [18]:
# Custom transformer for reading data
class DataReader(BaseEstimator, TransformerMixin):
    def __init__(self, filename):
        self.filename = filename
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return pd.read_csv(self.filename, encoding='ISO-8859-1')

# Custom transformer for cleaning data
class DataCleaner(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_clean = X.drop(columns=['Days for shipping (real)', 'Delivery Status', 'Late_delivery_risk', 
                                  'shipping date (DateOrders)', 'Benefit per order', 'Sales per customer', 'Category Id',
                                  'Order Profit Per Order', 'Order Item Discount', 'Order Item Total', 'Order Status', 
                                  'Customer Email', 'Customer Password', 'Latitude', 'Longitude', 'Product Description', 'Product Image',
                                  'Customer Fname', 'Customer Id', 'Customer Lname', 'Department Id',
                                  'Order Customer Id', 'Order Item Cardprod Id', 'Order Item Id',
                                  'Product Card Id', 'Product Category Id', 'Order Id', 'Customer Street',
                                  'Customer Zipcode', 'Order Zipcode', 'Order Item Product Price',
                                  'Product Price', 'Order Item Profit Ratio', 'Product Status'])
        X_clean['order date (DateOrders)'] = pd.to_datetime(X_clean['order date (DateOrders)'])
        X_clean['Year'] = X_clean['order date (DateOrders)'].dt.year
        X_clean['Month'] = X_clean['order date (DateOrders)'].dt.month
        X_clean.sort_values(by='order date (DateOrders)', inplace=True)
        X_clean.drop(columns=['order date (DateOrders)'], inplace=True)
        return X_clean

# Custom transformer for aggregating data
class DataAggregator(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        grouped_data = X.groupby(['Type', 'Category Name', 'Customer City', 'Customer Country',
                                  'Customer Segment', 'Customer State', 'Department Name', 'Market',
                                  'Order City', 'Order Country', 'Order Region', 'Order State',
                                  'Product Name', 'Shipping Mode', 'Year', 'Month'])
        aggregated_data = grouped_data.agg({
            'Sales': 'sum',
            'Days for shipment (scheduled)': 'mean', 
            'Order Item Discount Rate': 'mean',
            'Order Item Quantity': 'sum'
        }).reset_index()
        return aggregated_data

# Custom transformer for saving data
class DataSaver(BaseEstimator, TransformerMixin):
    def __init__(self, filename):
        self.filename = filename
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X.to_csv(self.filename, index=False)
        return X

# Define the pipeline
pipeline = Pipeline([
    ('read_data', DataReader(filename="../../data/raw/Q1_2015.csv")),
    ('clean_data', DataCleaner()),
    ('aggregate_data', DataAggregator()),
    ('save_data', DataSaver(filename="../../data/processed/Q1_2015_cleaned.csv"))
])

# Execute the pipeline
pipeline.fit_transform(None)


,Type,Category Name,Customer City,Customer Country,Customer Segment,Customer State,Department Name,Market,Order City,Order Country,Order Region,Order State,Product Name,Shipping Mode,Year,Month,Sales,Days for shipment (scheduled),Order Item Discount Rate,Order Item Quantity
0,CASH,Accessories,Bronx,EE. UU.,Consumer,NY,Outdoors,LATAM,Mejicanos,El Salvador,Central America,San Salvador,Team Golf Pittsburgh Steelers Putter Grip,First Class,2015,2,124.949997,1.0,0.000,5
1,CASH,Accessories,Brooklyn,EE. UU.,Home Office,NY,Outdoors,LATAM,La Romana,República Dominicana,Caribbean,La Romana,Team Golf Tennessee Volunteers Putter Grip,First Class,2015,2,74.970001,1.0,0.200,3
2,CASH,Accessories,Brownsville,EE. UU.,Corporate,TX,Outdoors,LATAM,San Salvador,El Salvador,Central America,San Salvador,Team Golf New England Patriots Putter Grip,Standard Class,2015,1,49.980000,4.0,0.250,2
3,CASH,Accessories,Buffalo,EE. UU.,Corporate,NY,Outdoors,LATAM,Santiago de los Caballeros,República Dominicana,Caribbean,Santiago de Chile,Team Golf St. Louis Cardinals Putter Grip,Standard Class,2015,1,124.949997,4.0,0.120,5
4,CASH,Accessories,Caguas,Puerto Rico,Consumer,PR,Outdoors,LATAM,Buenos Aires,Argentina,South America,Buenos Aires,Team Golf New England Patriots Putter Grip,Second Class,2015,3,74.970001,2.0,0.090,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13174,TRANSFER,Women's Apparel,West Covina,EE. UU.,Corporate,CA,Golf,LATAM,Santo Domingo,República Dominicana,Caribbean,Santo Domingo,Nike Men's Dri-FIT Victory Golf Polo,Standard Class,2015,2,200.000000,4.0,0.090,4
13175,TRANSFER,Women's Apparel,West Jordan,EE. UU.,Corporate,UT,Golf,LATAM,Lagos de Moreno,México,Central America,Jalisco,Nike Men's Dri-FIT Victory Golf Polo,Standard Class,2015,3,300.000000,4.0,0.155,6
13176,TRANSFER,Women's Apparel,West Jordan,EE. UU.,Corporate,UT,Golf,LATAM,Mejicanos,El Salvador,Central America,San Salvador,Nike Men's Dri-FIT Victory Golf Polo,Second Class,2015,2,150.000000,2.0,0.090,3
13177,TRANSFER,Women's Apparel,West Lafayette,EE. UU.,Consumer,IN,Golf,LATAM,Lima,Perú,South America,Lima (ciudad),Nike Men's Dri-FIT Victory Golf Polo,First Class,2015,1,200.000000,1.0,0.060,4
